In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as mno
from tqdm import tqdm
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
from tensorflow.keras import Sequential #khung 
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense 
import tensorflow as tf
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_ = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/cleaned_data_train_2702.csv')
print(train_.shape)
train_

(128657, 173)


,Unnamed: 0,age,elective_surgery,height,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,...,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,ethnicity_Asian,ethnicity_Caucasian,ethnicity_Hispanic,ethnicity_Native American,ethnicity_Other/Unknown,gender_M,hospital_admit_source_Chest Pain Center,hospital_admit_source_Direct Admit,hospital_admit_source_Emergency Department,hospital_admit_source_Floor,hospital_admit_source_ICU,hospital_admit_source_ICU to SDU,hospital_admit_source_Observation,hospital_admit_source_Operating Room,hospital_admit_source_Other,hospital_admit_source_Other Hospital,hospital_admit_source_Other ICU,hospital_admit_source_PACU,hospital_admit_source_Recovery Room,hospital_admit_source_Step-Down Unit (SDU),icu_admit_source_Floor,icu_admit_source_Operating Room / Recovery,icu_admit_source_Other Hospital,icu_admit_source_Other ICU,icu_stay_type_readmit,icu_stay_type_transfer,icu_type_CSICU,icu_type_CTICU,icu_type_Cardiac ICU,icu_type_MICU,icu_type_Med-Surg ICU,icu_type_Neuro ICU,icu_type_SICU,bmi_category,label
0,0,68.0,0,180.3,0.541667,0,73.9,2.300000,113.0,502.01,0,0,0.400000,31.000000,2.510000,0.608874,3.0,6.0,0.0,4.0,168.000000,118.0,27.400000,0,40.0,42.503378,42.503378,130.126947,7.352285,36.0,134.000000,39.3,1880.975174,0,14.100000,46.000000,32.000000,68.0,37.0,68.0,...,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1.0
1,1,77.0,0,160.0,0.927778,0,70.2,2.842762,108.0,203.01,0,0,1.012062,9.000000,0.560000,1.000000,1.0,3.0,0.0,1.0,145.000000,120.0,36.900000,0,46.0,37.000000,37.000000,51.000000,7.450000,33.0,145.000000,35.1,1681.810832,1,12.700000,79.773308,46.013596,95.0,31.0,95.0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,2,1.0
2,2,25.0,0,172.7,0.000694,0,95.3,2.842762,122.0,703.03,0,0,1.012062,24.040124,1.240601,0.581356,3.0,6.0,0.0,5.0,159.311340,102.0,32.152284,0,68.0,42.335091,42.335091,131.026038,7.351192,37.0,138.004869,36.7,1681.810832,0,12.337758,79.773308,46.013596,88.0,48.0,88.0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,3,0.0
3,3,81.0,1,165.1,0.000694,0,61.7,2.842762,203.0,1206.03,1,0,1.012062,24.040124,1.240601,0.600000,4.0,6.0,0.0,5.0,185.000000,114.0,25.900000,1,60.0,30.000000,30.000000,142.000000,7.390000,4.0,138.004869,34.8,1681.810832,1,8.000000,62.000000,30.000000,48.0,42.0,48.0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0.0
4,4,67.0,0,190.5,0.000694,0,100.0,2.932497,301.0,403.01,0,0,1.295918,13.000000,0.710000,0.608874,4.0,6.0,0.0,5.0,156.000000,113.0,44.200000,0,130.0,42.503378,42.503378,130.126947,7.352285,35.0,137.000000,36.6,1880.975174,0,10.900000,78.179184,46.954389,100.0,61.0,100.0,...,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128652,128652,50.0,0,175.3,0.023611,0,90.0,2.932497,110.0,104.01,0,0,1.295918,26.841166,1.517321,0.608874,4.0,6.0,0.0,5.0,158.405088,106.0,33.930284,0,120.0,42.503378,42.503378,130.126947,7.352285,50.0,137.822495,36.2,1751.155200,0,12.248268,78.179184,46.954389,102.0,82.0,102.0,...,0,0,0,0,0,0,1,0,0,0,1

## With all features

In [ ]:
y = train_.iloc[:,-1]
X = train_.iloc[:,1:-1]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1, stratify=y)
y.value_counts()/len(y)

0.0    0.782694
1.0    0.217306
Name: label, dtype: float64

In [ ]:
from imblearn.over_sampling import RandomOverSampler
over = RandomOverSampler(sampling_strategy=0.4)
X_over, y_over = over.fit_resample(X_train, y_train)
from imblearn.under_sampling import RandomUnderSampler
under = RandomUnderSampler(sampling_strategy=0.6)
X_under, y_under = under.fit_resample(X_over, y_over)

In [ ]:
from collections import Counter
print(Counter(y_under))
print(X_under.shape)
print(X_test.shape)

Counter({0.0: 53705, 1.0: 32223})
(85928, 171)
(25732, 171)


In [ ]:
X_under_train = X_under.reshape(85928,1,171,1)
X_test = X_test.to_numpy()
X_test = X_test.reshape(25732,1,171,1)

## Built CNN 

In [ ]:
#Part 1: Create a box
#Initialising the CNN - create an empty box
classifier= Sequential()
#Step 1- Convolution:
classifier.add(Conv2D(32,(1,3), input_shape=(1,171,1),activation='relu'))
#Step 2- Pooling:
classifier.add(MaxPooling2D(pool_size=(1,2)))
#Add a second convolutional layer
classifier.add(Conv2D(32,(1,3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(1,2)))
# Step 3: Flattening
classifier.add(Flatten()) # ma tran lam phang 
# Step 4: Fully connection
classifier.add(Dense(units=128, activation='relu')) #hidden layers # align with input_shape
classifier.add(Dense(units=1, activation='sigmoid')) # ANN
# Compiling the CNN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 1, 169, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 1, 84, 32)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 82, 32)         3104      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 41, 32)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1312)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               168064    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

In [ ]:
classifier.fit(X_under_train, y_under, validation_data=(X_test, y_test), epochs=15)

Epoch 1/15
2686/2686 [==============================] - 28s 10ms/step - loss: 1.0073 - accuracy: 0.7034 - val_loss: 0.4365 - val_accuracy: 0.8103
Epoch 2/15
2686/2686 [==============================] - 26s 10ms/step - loss: 0.5124 - accuracy: 0.7518 - val_loss: 0.4388 - val_accuracy: 0.7920
Epoch 3/15
2686/2686 [==============================] - 26s 10ms/step - loss: 0.5006 - accuracy: 0.7563 - val_loss: 0.4961 - val_accuracy: 0.7460
Epoch 4/15
2686/2686 [==============================] - 26s 10ms/step - loss: 0.4912 - accuracy: 0.7640 - val_loss: 0.4155 - val_accuracy: 0.8035
Epoch 5/15
2686/2686 [==============================] - 26s 10ms/step - loss: 0.4865 - accuracy: 0.7674 - val_loss: 0.4376 - val_accuracy: 0.7858
Epoch 6/15
2686/2686 [==============================] - 35s 13ms/step - loss: 0.4847 - accuracy: 0.7706 - val_loss: 0.4134 - val_accuracy: 0.8064
Epoch 7/15
2686/2686 [==============================] - 26s 10ms/step - loss: 0.4764 - accuracy: 0.7747 - val_loss: 0.4238 -

In [ ]:
predict_ = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/cleaned_data_test_2702.csv')
print(predict_.shape)
predict_

(10234, 172)


,Unnamed: 0,age,elective_surgery,height,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,...,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,ethnicity_Asian,ethnicity_Caucasian,ethnicity_Hispanic,ethnicity_Native American,ethnicity_Other/Unknown,gender_M,hospital_admit_source_Chest Pain Center,hospital_admit_source_Direct Admit,hospital_admit_source_Emergency Department,hospital_admit_source_Floor,hospital_admit_source_ICU,hospital_admit_source_ICU to SDU,hospital_admit_source_Observation,hospital_admit_source_Operating Room,hospital_admit_source_Other,hospital_admit_source_Other Hospital,hospital_admit_source_Other ICU,hospital_admit_source_PACU,hospital_admit_source_Recovery Room,hospital_admit_source_Step-Down Unit (SDU),icu_admit_source_Floor,icu_admit_source_Operating Room / Recovery,icu_admit_source_Other Hospital,icu_admit_source_Other ICU,icu_stay_type_readmit,icu_stay_type_transfer,icu_type_CSICU,icu_type_CTICU,icu_type_Cardiac ICU,icu_type_MICU,icu_type_Med-Surg ICU,icu_type_Neuro ICU,icu_type_SICU,bmi_category
0,128657,72.0,0,152.4,0.015278,0,72.5,2.800000,110.0,104.01,0,0,1.900000,44.000000,1.490000,0.581356,4.0,6.0,0.0,5.0,97.000000,38.0,39.900000,0,54.0,42.335091,42.335091,131.026038,7.351192,31.0,130.000000,36.4,1681.810832,0,5.400000,79.773308,46.013596,104.0,40.0,104.0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2
1,128658,86.0,0,175.3,0.000000,0,72.5,2.842762,117.0,106.01,0,0,1.012062,19.000000,0.920000,0.581356,4.0,6.0,0.0,5.0,73.000000,116.0,32.152284,0,41.0,42.335091,42.335091,131.026038,7.351192,53.0,142.000000,36.3,1681.810832,0,12.337758,79.773308,46.013596,101.0,27.0,101.0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2
2,128659,72.0,0,162.6,0.003472,0,72.5,3.200000,302.0,109.01,0,0,0.900000,24.000000,0.770000,0.581356,4.0,6.0,0.0,5.0,84.000000,104.0,34.800000,0,117.0,42.335091,42.335091,131.026038,7.351192,38.0,137.000000,36.4,1681.810832,0,4.600000,79.773308,46.013596,97.0,61.0,97.0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,2
3,128660,66.0,0,177.8,0.884028,0,85.7,2.932497,113.0,501.05,0,0,1.295918,26.841166,1.517321,0.608874,4.0,6.0,0.0,4.0,158.405088,110.0,33.930284,0,61.0,42.503378,42.503378,130.126947,7.352285,56.0,137.822495,36.6,1880.975174,1,12.248268,78.179184,46.954389,77.0,51.0,77.0,...,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,2
4,128661,89.0,0,170.2,0.013194,0,85.7,3.400000,117.0,106.01,0,0,0.400000,26.000000,1.500000,0.608874,4.0,6.0,0.0,5.0,99.000000,34.0,33.000000,0,136.0,42.503378,42.503378,130.126947,7.352285,35.0,133.000000,36.5,1880.975174,0,5.200000,78.179184,46.954389,88.0,49.0,88.0,...,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10229,138886,36.0,0,170.1,1.696528,0,108.6,2.842762,305.0,901.02,0,0,1.012062,24.040124,1.240601,0.581356,3.0,6.0,0.0,5.0,159.311340,111.0,29.000000,0,127.0,42.335091,42.335091,131.026038,7.351192,45.0,138.004869,36.5,2284.000000,0,7.200000,79.773308,46.013596,98.0,68.0,98.0,...,0,0,0,0,0

In [ ]:
X_predict = predict_.iloc[:,1:]
X_predict = X_predict.to_numpy()
X_predict = X_predict.reshape(10234,1,171,1)

Function to check result with input prob which classifier one patient be or not be sick

In [ ]:
def check(X_predict, X_train, y_train, X_test, y_test, prob):
  testPredict = classifier.predict(X_predict)
  testTrain = classifier.predict(X_train)
  testTest = classifier.predict(X_test)
  m = len(testPredict[testPredict >= prob])
  a = len(testTrain[testTrain >= prob]) / len(y_train[y_train == 1])
  b = len(testTest[testTest >= prob]) / len(y_test[y_test == 1])
  print(prob, m, a, b)

In [ ]:
i = 0.3
while i <= 0.4:
  check(X_predict, X_under_train, y_under, X_test, y_test, i)
  i += 0.01

0.3 5135 1.5538900785153462 2.344241773962804
0.31 5032 1.529745833721255 2.300429184549356
0.32 4910 1.5062222636005338 2.2562589413447784
0.33 4821 1.481674580268752 2.206902718168813
0.34 4722 1.4593613257611022 2.163805436337625
0.35000000000000003 4633 1.437451509791143 2.1187410586552216
0.36000000000000004 4526 1.4148279179468082 2.0804721030042916
0.37000000000000005 4433 1.3919250224994568 2.03826895565093
0.38000000000000006 4349 1.369301430655122 2.001251788268956
0.39000000000000007 4260 1.3483536604288862 1.9569027181688126


In [ ]:
result = classifier.predict(X_predict)
result = result.reshape(10234)
print(len(result[result >= 0.28]))
for i in range(len(result)):
  if result[i] >= 0.28:
    result[i] = 1
  else: result[i] = 0
print(len(result[result == 1]))

5344
5344


In [ ]:
q = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/submission_11.csv')
encounter_id = q['encounter_id']

In [ ]:
dataset = pd.DataFrame({'encounter_id': encounter_id, 'diabetes_mellitus': result})
dataset.to_csv('/content/gdrive/MyDrive/Colab Notebooks/under28.csv')
dataset

,encounter_id,diabetes_mellitus
0,144740,0.0
1,141990,0.0
2,142038,1.0
3,138628,0.0
4,141682,1.0
...,...,...
10229,143750,0.0
10230,143813,0.0
10231,137126,1.0
10232,135652,0.0


In [ ]:
len(dataset['diabetes_mellitus'][dataset['diabetes_mellitus'] == 1])

5344

# **Top 10 features**

In [ ]:
top10 = ['d1_glucose_max', 'glucose_apache', 'h1_glucose_max', 'bmi_category', 'h1_glucose_min', 'd1_glucose_min', 'age', 
         'd1_creatinine_min', 'd1_bun_min', 'd1_bun_max', 'd1_creatinine_max', 'creatinine_apache']
y = train_.iloc[:,-1]
X = train_.loc[:,top10]
X

,d1_glucose_max,glucose_apache,h1_glucose_max,bmi_category,h1_glucose_min,d1_glucose_min,age,d1_creatinine_min,d1_bun_min,d1_bun_max,d1_creatinine_max,creatinine_apache
0,168.000000,168.000000,164.830653,1,156.400769,109.000000,68.0,2.230000,30.000000,31.000000,2.510000,2.510000
1,145.000000,145.000000,145.000000,2,143.000000,128.000000,77.0,0.560000,9.000000,11.000000,0.710000,0.560000
2,172.667436,159.311340,167.721828,3,159.128092,114.567355,25.0,1.151752,22.126190,23.966194,1.256880,1.240601
3,185.000000,185.000000,167.721828,1,159.128092,88.000000,81.0,1.151752,22.126190,23.966194,1.256880,1.240601
4,156.000000,156.000000,164.830653,2,156.400769,125.000000,67.0,0.710000,13.000000,13.000000,0.710000,0.710000
...,...,...,...,...,...,...,...,...,...,...,...,...
128652,172.186040,158.405088,164.830653,2,156.400769,114.935354,50.0,1.393122,24.628016,26.682274,1.525476,1.517321
128653,139.000000,139.000000,167.721828,2,159.128092,139.000000,79.0,1.070000,32.000000,32.000000,1.070000,1.070000
128654,346.000000,346.000000,174.909719,3,165.116524,55.000000,73.0,2.410000,30.000000,31.000000,2.650000,2.430000
128655,156.000000,137.000000,164.830653,1,156.400769,137.000000,81.0,2.210000,45.000000,45.000000,2.210000,1.517321


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1, stratify=y)
over = RandomOverSampler(sampling_strategy=0.4)
X_over, y_over = over.fit_resample(X_train, y_train)
under = RandomUnderSampler(sampling_strategy=0.6)
X_under, y_under = under.fit_resample(X_over, y_over)
print(Counter(y_under))

Counter({0.0: 53705, 1.0: 32223})


In [ ]:
print(X_under.shape)
print(X_test.shape)

(85928, 12)
(25732, 12)


In [ ]:
X_under_train = X_under.reshape(85928,1,12,1)
X_test = X_test.to_numpy()
X_test = X_test.reshape(25732,1,12,1)

In [ ]:
#Part 1: Create a box
#Initialising the CNN - create an empty box
classifier= Sequential()
#Step 1- Convolution:
classifier.add(Conv2D(32,(1,3), input_shape=(1,12,1),activation='relu'))
#Step 2- Pooling:
classifier.add(MaxPooling2D(pool_size=(1,2)))
#Add a second convolutional layer
classifier.add(Conv2D(32,(1,3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(1,2)))
# Step 3: Flattening
classifier.add(Flatten()) # ma tran lam phang 
# Step 4: Fully connection
classifier.add(Dense(units=128, activation='relu')) #hidden layers # align with input_shape
classifier.add(Dense(units=1, activation='sigmoid')) # ANN
# Compiling the CNN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classifier.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 1, 10, 32)         128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 1, 5, 32)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 1, 3, 32)          3104      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 1, 1, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               4224      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                

In [ ]:
classifier.fit(X_under_train, y_under, validation_data=(X_test, y_test), epochs=15)

Epoch 1/15
2686/2686 [==============================] - 7s 2ms/step - loss: 0.6808 - accuracy: 0.6917 - val_loss: 0.6337 - val_accuracy: 0.6533
Epoch 2/15
2686/2686 [==============================] - 6s 2ms/step - loss: 0.5493 - accuracy: 0.7277 - val_loss: 0.5026 - val_accuracy: 0.7776
Epoch 3/15
2686/2686 [==============================] - 6s 2ms/step - loss: 0.5362 - accuracy: 0.7369 - val_loss: 0.4387 - val_accuracy: 0.7941
Epoch 4/15
2686/2686 [==============================] - 6s 2ms/step - loss: 0.5207 - accuracy: 0.7449 - val_loss: 0.4896 - val_accuracy: 0.7633
Epoch 5/15
2686/2686 [==============================] - 6s 2ms/step - loss: 0.5171 - accuracy: 0.7459 - val_loss: 0.4542 - val_accuracy: 0.7814
Epoch 6/15
2686/2686 [==============================] - 6s 2ms/step - loss: 0.5125 - accuracy: 0.7493 - val_loss: 0.4540 - val_accuracy: 0.7863
Epoch 7/15
2686/2686 [==============================] - 6s 2ms/step - loss: 0.5112 - accuracy: 0.7497 - val_loss: 0.4841 - val_accuracy:

In [ ]:
X_predict = predict_.loc[:,top10]
X_predict = X_predict.to_numpy()
X_predict = X_predict.reshape(10234,1,12,1)

In [ ]:
result = classifier.predict(X_predict)

In [ ]:
result = classifier.predict(X_predict)
result = result.reshape(10234)
print(len(result[result >= 0.28]))
for i in range(len(result)):
  if result[i] >= 0.28:
    result[i] = 1
  else: result[i] = 0
print(len(result[result == 1]))

3796
3796


In [ ]:
dataset = pd.DataFrame({'encounter_id': encounter_id, 'diabetes_mellitus': result})
dataset.to_csv('/content/gdrive/MyDrive/Colab Notebooks/under28_10features.csv')
dataset

,encounter_id,diabetes_mellitus
0,144740,0.0
1,141990,0.0
2,142038,0.0
3,138628,0.0
4,141682,1.0
...,...,...
10229,143750,0.0
10230,143813,0.0
10231,137126,0.0
10232,135652,0.0


In [ ]:
len(dataset['diabetes_mellitus'][dataset['diabetes_mellitus'] == 1])

3796